# **Problema clasificación con RNA y Keras**
* RNA = Redes Neuronales Artificiales

In [77]:
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import random

import warnings

# Para ignorar todas las advertencias
warnings.filterwarnings("ignore")

In [78]:
from numpy.random import seed
seed(1)

In [79]:
# Cargamos el conjunto de datos
# os.chdir (os.getcwd())
df = pd.read_csv(r"data/diabetes.csv", sep=",", header=None)

In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       768 non-null    int64  
 1   1       768 non-null    int64  
 2   2       768 non-null    int64  
 3   3       768 non-null    int64  
 4   4       768 non-null    int64  
 5   5       768 non-null    float64
 6   6       768 non-null    float64
 7   7       768 non-null    int64  
 8   8       768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [81]:
df.head()

,0,1,2,3,4,5,6,7,8
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Significado de las columnas:

 0. Number of times pregnant
 1. Plasma glucose concentration a 2 hours in an oral glucose tolerance test
 2. Diastolic blood pressure (mm Hg)
 3. Triceps skin fold thickness (mm)
 4. 2-Hour serum insulin (mu U/ml)
 5. Body mass index (weight in kg/(height in m)^2)
 6. Diabetes pedigree function
 7. Age (years)
 8. **Variable clase (Tiene diabetes)**  ———> VARIABLE OBJETIVO

In [82]:
# Dividimos los datos en X e y
X = df.iloc[:,0:8]
y = df.iloc[:,8]

In [83]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
                                        X,
                                        y,
                                        train_size   = 0.7,
                                        random_state = 123,
                                        shuffle      = True
                                    )

### Definimos el modelo Keras

In [84]:
# Para obtener todos el mismo resultado debemos añadir una semilla
tf.random.set_seed(1) 
np.random.seed(1)
random.seed(1)
seed(1)

In [89]:
# Definiremos el modelo como una secuencia de capas.
# Usaremos el modelo secuencial de manera que podamos ir añadiendo capas hasta estar contentos con la arquitectura desarrollada.
model = Sequential()

# Partimos de un sistema con 8 variables por lo que nuestra primera capa acomodará dichas variables
# En la primera capa oculta usaremos 12 neuronas y una función de activación ReLU
# En la segunda capa oculta usaremos 8 neuronas y una función de activación ReLU
# Finalmente en la de salida una neurona y función de activación sigmoide
model.add(Dense(12, activation='relu'))
# model.add(Dense(12, input_dim=8, activation='relu')) # Me da WARNING si pongo el input_dim
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Nota: Fíjate que el total de neuronas de entrada, lo definimos en la primera capa con input_dim


### Compilamos el modelo

In [90]:
# OPCIÓN DE COMPILACIÓN: A

# La compilación usa (internamente) librerías numéricas muy eficientes como TensorFlow además de comprobar si tenemos GPU 
# o sólo CPU
# Es necesario definir la función de pérdida que vamos a minimizar (optimizar).  Para este caso minimizaremos 
# Binary Cross Entropy puesto que funciona bien para problemas binarios de clasificación.
# Como métrica (al ser clasificación) usaremos la precisión.
# Como optimizador, usaremos el algoritmo "adam" ya que ofrece buenos resultados en un amplio abanico de problemas y 
# además de manera rápida

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              #loss="binary_crossentropy", por que es una clasificación binaria
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False, # usar cuando valores de predicción sean [0,1]
                                                      label_smoothing=0.0,
                                                      axis=-1,
                                                    #   reduction="auto", # ME DA ERROR si le pongo AUTO
                                                      name="binary_crossentropy"), 
              metrics=['accuracy']) # La métrica eleginda


In [11]:
# OPCIÓN DE COMPILACIÓN: B

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.BinaryCrossentropy(
        from_logits=False,  # Usar cuando las predicciones están en el rango [0, 1]
        label_smoothing=0.0,
        axis=-1,
        reduction='sum_over_batch_size',  # AUTO nos da erro. Cambiado a un valor válido.
        name="binary_crossentropy" 
    ),
    metrics=['accuracy']  # Métrica de precisión
)

### Entrenamos el modelo

In [91]:
# seed(1)
# random.set_seed(1)
# El argumento batch_size, nos permite definir el número de filas que se considerarán, antes de que los pesos del
# modelo se reajusten dentro de cada ciclo.
model.fit(X_train, y_train, epochs=150, batch_size=10)

# En el aprendizaje automático, entrenar modelos de aprendizaje profundo con grandes conjuntos de datos puede 
# requerir mucha memoria y recursos computacionales. El uso de un tamaño de lote adecuado puede ayudar a evitar 
# problemas de memoria y a mejorar la velocidad de entrenamiento.

# Por ejemplo, si tenemos un conjunto de datos de 1000 muestras y establecemos batch_size=10, esto significa 
# que se utilizarán 10 muestras a la vez para actualizar los pesos del modelo en cada iteración del entrenamiento. 
# Por lo tanto, el proceso de entrenamiento se dividirá en 100 iteraciones, una para cada lote de 10 muestras.

# El tamaño de lote adecuado depende del conjunto de datos y del modelo. En general, un tamaño de lote más grande
# puede proporcionar un mejor rendimiento de entrenamiento, pero requiere más memoria. Por otro lado, un tamaño de
# lote más pequeño puede proporcionar una mejor generalización del modelo, pero puede requerir más iteraciones para
# converger a una solución óptima.

# En resumen, batch_size es un parámetro importante en el proceso de entrenamiento de modelos de aprendizaje profundo en Keras y su elección puede afectar el rendimiento del modelo y la velocidad de entrenamiento.

Epoch 1/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6059 - loss: 4.1260
Epoch 2/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6364 - loss: 2.6765
Epoch 3/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6280 - loss: 2.0559
Epoch 4/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6224 - loss: 1.6272
Epoch 5/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6422 - loss: 1.3417
Epoch 6/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 943us/step - accuracy: 0.6636 - loss: 1.1553
Epoch 7/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6571 - loss: 1.0223
Epoch 8/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6515 - loss: 0.9310
Epoch 9/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6441 - loss: 0.8759
Epoch 10/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6584 - loss: 0.8312
Epoch 11/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6665 - loss: 0.7981
Epoch 12/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accurac

### Evaluamos el modelo

In [92]:
# Con la red neuronal entrenada, ahora debemos evaluar cómo ha funcionado.
_, accuracy = model.evaluate(X_train, y_train)
print('Accuracy: %.2f' % (accuracy*100))

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 948us/step - accuracy: 0.7289 - loss: 0.5305
Accuracy: 77.28


In [14]:
# No es un mal resultado, tenemos una precisión de más del 75%

### Predicciones

In [15]:
predicciones = model.predict(X_test)

# La función sigmoide nos devueve los resultados en formato probabilidad.
# Convertimos los mismos a casos, tomando como umbral 0.5
y_pred = (predicciones > 0.5).astype(int)
y_pred

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


array([[1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
    

In [16]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[123,  20],
       [ 48,  40]])

---

# **Ejercicios**

## **1.Configura la red neuronal para que trabaje con 3 capas.** 
- La primera con 15 neuronas y función de activación sigmoide. 
- La segunda con 10 neuronas y función de activación sigmoide.
- La tercera con 8 neuronas y función de activación ReLU.
- Una capa de salida con 1 neurona y función de activación sigmoide.

In [17]:
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import random

In [18]:
# Cargamos el conjunto de datos
# os.chdir (os.getcwd())
df = pd.read_csv(r"data/diabetes.csv", sep=",", header=None)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       768 non-null    int64  
 1   1       768 non-null    int64  
 2   2       768 non-null    int64  
 3   3       768 non-null    int64  
 4   4       768 non-null    int64  
 5   5       768 non-null    float64
 6   6       768 non-null    float64
 7   7       768 non-null    int64  
 8   8       768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [20]:
df.head()

,0,1,2,3,4,5,6,7,8
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [21]:
# Dividimos los datos en X e y
X = df.iloc[:,0:8]
y = df.iloc[:,8]

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
                                        X,
                                        y,
                                        train_size   = 0.7,
                                        random_state = 123,
                                        shuffle      = True
                                    )

In [23]:
tf.random.set_seed(1) 

In [24]:
# Definiremos el modelo como una secuencia de capas.
# Usaremos el modelo secuencial de manera que podamos ir añadiendo capas hasta estar contentos con la arquitectura desarrollada.
model = Sequential()

# Partimos de un sistema con 8 variables por lo que nuestra primera capa acomodará dichas variables
# En la primera capa oculta usaremos 15 neuronas y una función de activación Sigmoid
# En la primera capa oculta usaremos 10 neuronas y una función de activación Sigmoid
# En la segunda capa oculta usaremos 8 neuronas y una función de activación ReLU
# Finalmente en la de salida una neurona y función de activación sigmoide
model.add(Dense(15, activation='sigmoid'))
model.add(Dense(10, activation='sigmoid'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Nota: Fíjate que el total de neuronas de entrada, lo definimos en la primera capa con input_dim


In [25]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.BinaryCrossentropy(
        from_logits=False,  # Usar cuando las predicciones están en el rango [0, 1]
        label_smoothing=0.0,
        axis=-1,
        reduction='sum_over_batch_size',  # Cambiado a un valor válido
        name="binary_crossentropy" 
    ),
    metrics=['accuracy']  # Métrica de precisión
)

Entrenamos modelo

In [26]:
# seed(1)
# random.set_seed(1)
# El argumento batch_size, nos permite definir el número de filas que se considerarán, antes de que los pesos del
# modelo se reajusten dentro de cada ciclo.
model.fit(X_train, y_train, epochs=150, batch_size=10)

# En el aprendizaje automático, entrenar modelos de aprendizaje profundo con grandes conjuntos de datos puede 
# requerir mucha memoria y recursos computacionales. El uso de un tamaño de lote adecuado puede ayudar a evitar 
# problemas de memoria y a mejorar la velocidad de entrenamiento.

# Por ejemplo, si tenemos un conjunto de datos de 1000 muestras y establecemos batch_size=10, esto significa 
# que se utilizarán 10 muestras a la vez para actualizar los pesos del modelo en cada iteración del entrenamiento. 
# Por lo tanto, el proceso de entrenamiento se dividirá en 100 iteraciones, una para cada lote de 10 muestras.

# El tamaño de lote adecuado depende del conjunto de datos y del modelo. En general, un tamaño de lote más grande
# puede proporcionar un mejor rendimiento de entrenamiento, pero requiere más memoria. Por otro lado, un tamaño de
# lote más pequeño puede proporcionar una mejor generalización del modelo, pero puede requerir más iteraciones para
# converger a una solución óptima.

# En resumen, batch_size es un parámetro importante en el proceso de entrenamiento de modelos de aprendizaje profundo en Keras y su elección puede afectar el rendimiento del modelo y la velocidad de entrenamiento.

Epoch 1/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 870us/step - accuracy: 0.6568 - loss: 0.6483
Epoch 2/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 945us/step - accuracy: 0.6572 - loss: 0.6420
Epoch 3/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 714us/step - accuracy: 0.6572 - loss: 0.6366
Epoch 4/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 794us/step - accuracy: 0.6572 - loss: 0.6336
Epoch 5/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 811us/step - accuracy: 0.6572 - loss: 0.6315
Epoch 6/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 772us/step - accuracy: 0.6572 - loss: 0.6304
Epoch 7/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 796us/step - accuracy: 0.6572 - loss: 0.6284
Epoch 8/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6572 - loss: 0.6267
Epoch 9/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6599 - loss: 0.6252
Epoch 10/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6594 - loss: 0.6231
Epoch 11/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6599 - loss: 0.6219
Epoch 12/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/st

#### Evaluamos modelo

In [27]:
# Con la red neuronal entrenada, ahora debemos evaluar cómo ha funcionado.
_, accuracy = model.evaluate(X_train, y_train)
print('Accuracy: %.2f' % (accuracy*100))

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7338 - loss: 0.5217
Accuracy: 73.93


#### Predicciones

In [28]:
predicciones = model.predict(X_test)

# La función sigmoide nos devueve los resultados en formato probabilidad.
# Convertimos los mismos a casos, tomando como umbral 0.5
y_pred = (predicciones > 0.5).astype(int)
y_pred

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 


array([[0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
    

In [29]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[114,  29],
       [ 52,  36]])

---

## **2.Configura la red neuronal para que trabaje con las 3 mismas capas del ejemplo inicial, pero esta vez usa como función de activación de la capa de salida 'softmax'**

In [30]:
# Definiremos el modelo como una secuencia de capas.
# Usaremos el modelo secuencial de manera que podamos ir añadiendo capas hasta estar contentos con la arquitectura desarrollada.
model = Sequential()

# Partimos de un sistema con 8 variables por lo que nuestra primera capa acomodará dichas variables
# En la primera capa oculta usaremos 15 neuronas y una función de activación Sigmoid
# En la primera capa oculta usaremos 10 neuronas y una función de activación Sigmoid
# En la segunda capa oculta usaremos 8 neuronas y una función de activación ReLU
# Finalmente en la de salida una neurona y función de activación sigmoide
model.add(Dense(15, activation='sigmoid'))
model.add(Dense(10, activation='sigmoid'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='softmax'))

# Nota: Fíjate que el total de neuronas de entrada, lo definimos en la primera capa con input_dim


In [31]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.BinaryCrossentropy(
        from_logits=False,  # Usar cuando las predicciones están en el rango [0, 1]
        label_smoothing=0.0,
        axis=-1,
        reduction='sum_over_batch_size',  # Cambiado a un valor válido
        name="binary_crossentropy" 
    ),
    metrics=['accuracy']  # Métrica de precisión
)

In [32]:
# seed(1)
# random.set_seed(1)
# El argumento batch_size, nos permite definir el número de filas que se considerarán, antes de que los pesos del
# modelo se reajusten dentro de cada ciclo.
model.fit(X_train, y_train, epochs=150, batch_size=10)

# En el aprendizaje automático, entrenar modelos de aprendizaje profundo con grandes conjuntos de datos puede 
# requerir mucha memoria y recursos computacionales. El uso de un tamaño de lote adecuado puede ayudar a evitar 
# problemas de memoria y a mejorar la velocidad de entrenamiento.

# Por ejemplo, si tenemos un conjunto de datos de 1000 muestras y establecemos batch_size=10, esto significa 
# que se utilizarán 10 muestras a la vez para actualizar los pesos del modelo en cada iteración del entrenamiento. 
# Por lo tanto, el proceso de entrenamiento se dividirá en 100 iteraciones, una para cada lote de 10 muestras.

# El tamaño de lote adecuado depende del conjunto de datos y del modelo. En general, un tamaño de lote más grande
# puede proporcionar un mejor rendimiento de entrenamiento, pero requiere más memoria. Por otro lado, un tamaño de
# lote más pequeño puede proporcionar una mejor generalización del modelo, pero puede requerir más iteraciones para
# converger a una solución óptima.

# En resumen, batch_size es un parámetro importante en el proceso de entrenamiento de modelos de aprendizaje profundo en Keras y su elección puede afectar el rendimiento del modelo y la velocidad de entrenamiento.

Epoch 1/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 964us/step - accuracy: 0.3428 - loss: 0.6696
Epoch 2/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 895us/step - accuracy: 0.3428 - loss: 0.6418
Epoch 3/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 739us/step - accuracy: 0.3428 - loss: 0.6392
Epoch 4/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3428 - loss: 0.6377
Epoch 5/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3428 - loss: 0.6359
Epoch 6/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 790us/step - accuracy: 0.3428 - loss: 0.6343
Epoch 7/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 835us/step - accuracy: 0.3428 - loss: 0.6326
Epoch 8/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3428 - loss: 0.6309
Epoch 9/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 838us/step - accuracy: 0.3428 - loss: 0.6291
Epoch 10/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 836us/step - accuracy: 0.3428 - loss: 0.6272
Epoch 11/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 850us/step - accuracy: 0.3423 - loss: 0.6247
Epoch 12/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 861u

In [33]:
#### EVALUAMOS MODELO

# Con la red neuronal entrenada, ahora debemos evaluar cómo ha funcionado.
_, accuracy = model.evaluate(X_train, y_train)
print('Accuracy: %.2f' % (accuracy*100))

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3470 - loss: 0.5721
Accuracy: 33.52


In [34]:
#### PREDICCIONES ####

predicciones = model.predict(X_test)

# La función sigmoide nos devueve los resultados en formato probabilidad.
# Convertimos los mismos a casos, tomando como umbral 0.5
y_pred = (predicciones > 0.5).astype(int)
y_pred

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
    

In [35]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  0, 143],
       [  0,  88]])

---

## **3.En compile, cambia la configuración del optimizer, de manera que en vez de Adam, usemos esta vez SGD:**
optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.1, nesterov=False, name='SGD')

In [36]:
# Definiremos el modelo como una secuencia de capas.
# Usaremos el modelo secuencial de manera que podamos ir añadiendo capas hasta estar contentos con la arquitectura desarrollada.
model = Sequential()

# Partimos de un sistema con 8 variables por lo que nuestra primera capa acomodará dichas variables
# En la primera capa oculta usaremos 15 neuronas y una función de activación Sigmoid
# En la primera capa oculta usaremos 10 neuronas y una función de activación Sigmoid
# En la segunda capa oculta usaremos 8 neuronas y una función de activación ReLU
# Finalmente en la de salida una neurona y función de activación sigmoide
model.add(Dense(15, activation='sigmoid'))
model.add(Dense(10, activation='sigmoid'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='softmax'))

# Nota: Fíjate que el total de neuronas de entrada, lo definimos en la primera capa con input_dim

In [37]:
model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.1, nesterov=False, name='SGD'),
    loss=tf.keras.losses.BinaryCrossentropy(
        from_logits=False,  # Usar cuando las predicciones están en el rango [0, 1]
        label_smoothing=0.0,
        axis=-1,
        reduction='sum_over_batch_size',  # Cambiado a un valor válido
        name="SGD" 
    ),
    metrics=['accuracy']  # Métrica de precisión
)

In [38]:
# seed(1)
# random.set_seed(1)
# El argumento batch_size, nos permite definir el número de filas que se considerarán, antes de que los pesos del
# modelo se reajusten dentro de cada ciclo.
model.fit(X_train, y_train, epochs=150, batch_size=10)

# En el aprendizaje automático, entrenar modelos de aprendizaje profundo con grandes conjuntos de datos puede 
# requerir mucha memoria y recursos computacionales. El uso de un tamaño de lote adecuado puede ayudar a evitar 
# problemas de memoria y a mejorar la velocidad de entrenamiento.

# Por ejemplo, si tenemos un conjunto de datos de 1000 muestras y establecemos batch_size=10, esto significa 
# que se utilizarán 10 muestras a la vez para actualizar los pesos del modelo en cada iteración del entrenamiento. 
# Por lo tanto, el proceso de entrenamiento se dividirá en 100 iteraciones, una para cada lote de 10 muestras.

# El tamaño de lote adecuado depende del conjunto de datos y del modelo. En general, un tamaño de lote más grande
# puede proporcionar un mejor rendimiento de entrenamiento, pero requiere más memoria. Por otro lado, un tamaño de
# lote más pequeño puede proporcionar una mejor generalización del modelo, pero puede requerir más iteraciones para
# converger a una solución óptima.

# En resumen, batch_size es un parámetro importante en el proceso de entrenamiento de modelos de aprendizaje profundo en Keras y su elección puede afectar el rendimiento del modelo y la velocidad de entrenamiento.

Epoch 1/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.3428 - loss: 0.7085
Epoch 2/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3428 - loss: 0.6544
Epoch 3/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3426 - loss: 0.6432
Epoch 4/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 985us/step - accuracy: 0.3428 - loss: 0.6409
Epoch 5/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 903us/step - accuracy: 0.3428 - loss: 0.6403
Epoch 6/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 993us/step - accuracy: 0.3428 - loss: 0.6398
Epoch 7/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 960us/step - accuracy: 0.3428 - loss: 0.6395
Epoch 8/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 871us/step - accuracy: 0.3428 - loss: 0.6392
Epoch 9/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 900us/step - accuracy: 0.3432 - loss: 0.6394
Epoch 10/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 943us/step - accuracy: 0.3428 - loss: 0.6388
Epoch 11/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 871us/step - accuracy: 0.3428 - loss: 0.6386
Epoch 12/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 865u

In [39]:
#### EVALUAMOS MODELO

# Con la red neuronal entrenada, ahora debemos evaluar cómo ha funcionado.
_, accuracy = model.evaluate(X_train, y_train)
print('Accuracy: %.2f' % (accuracy*100))

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3470 - loss: 0.6010
Accuracy: 33.52


In [40]:
#### PREDICCIONES ####

predicciones = model.predict(X_test)

# La función sigmoide nos devueve los resultados en formato probabilidad.
# Convertimos los mismos a casos, tomando como umbral 0.5
y_pred = (predicciones > 0.5).astype(int)
y_pred

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
    

In [41]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  0, 143],
       [  0,  88]])

## **3.1.Cambia ahora la función de activación de la capa de salida a sigmoide**

In [42]:
# Definiremos el modelo como una secuencia de capas.
# Usaremos el modelo secuencial de manera que podamos ir añadiendo capas hasta estar contentos con la arquitectura desarrollada.
model = Sequential()

# Partimos de un sistema con 8 variables por lo que nuestra primera capa acomodará dichas variables
# En la primera capa oculta usaremos 15 neuronas y una función de activación Sigmoid
# En la primera capa oculta usaremos 10 neuronas y una función de activación Sigmoid
# En la segunda capa oculta usaremos 8 neuronas y una función de activación ReLU
# Finalmente en la de salida una neurona y función de activación sigmoide
model.add(Dense(15, activation='sigmoid'))
model.add(Dense(10, activation='sigmoid'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Nota: Fíjate que el total de neuronas de entrada, lo definimos en la primera capa con input_dim

In [43]:
model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.1, nesterov=False, name='SGD'),
    loss=tf.keras.losses.BinaryCrossentropy(
        from_logits=False,  # Usar cuando las predicciones están en el rango [0, 1]
        label_smoothing=0.0,
        axis=-1,
        reduction='sum_over_batch_size',  # Cambiado a un valor válido
        name="SGD" 
    ),
    metrics=['accuracy']  # Métrica de precisión
)

In [44]:
# seed(1)
# random.set_seed(1)
# El argumento batch_size, nos permite definir el número de filas que se considerarán, antes de que los pesos del
# modelo se reajusten dentro de cada ciclo.
model.fit(X_train, y_train, epochs=150, batch_size=10)

# En el aprendizaje automático, entrenar modelos de aprendizaje profundo con grandes conjuntos de datos puede 
# requerir mucha memoria y recursos computacionales. El uso de un tamaño de lote adecuado puede ayudar a evitar 
# problemas de memoria y a mejorar la velocidad de entrenamiento.

# Por ejemplo, si tenemos un conjunto de datos de 1000 muestras y establecemos batch_size=10, esto significa 
# que se utilizarán 10 muestras a la vez para actualizar los pesos del modelo en cada iteración del entrenamiento. 
# Por lo tanto, el proceso de entrenamiento se dividirá en 100 iteraciones, una para cada lote de 10 muestras.

# El tamaño de lote adecuado depende del conjunto de datos y del modelo. En general, un tamaño de lote más grande
# puede proporcionar un mejor rendimiento de entrenamiento, pero requiere más memoria. Por otro lado, un tamaño de
# lote más pequeño puede proporcionar una mejor generalización del modelo, pero puede requerir más iteraciones para
# converger a una solución óptima.

# En resumen, batch_size es un parámetro importante en el proceso de entrenamiento de modelos de aprendizaje profundo en Keras y su elección puede afectar el rendimiento del modelo y la velocidad de entrenamiento.

Epoch 1/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 897us/step - accuracy: 0.3428 - loss: 0.8490
Epoch 2/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4290 - loss: 0.7035
Epoch 3/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6572 - loss: 0.6670
Epoch 4/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6572 - loss: 0.6559
Epoch 5/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6573 - loss: 0.6512
Epoch 6/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6572 - loss: 0.6481
Epoch 7/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 921us/step - accuracy: 0.6572 - loss: 0.6459
Epoch 8/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6577 - loss: 0.6441
Epoch 9/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6572 - loss: 0.6431
Epoch 10/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 780us/step - accuracy: 0.6585 - loss: 0.6412
Epoch 11/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 686us/step - accuracy: 0.6572 - loss: 0.6411
Epoch 12/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 880us/step -

In [45]:
#### EVALUAMOS MODELO

# Con la red neuronal entrenada, ahora debemos evaluar cómo ha funcionado.
_, accuracy = model.evaluate(X_train, y_train)
print('Accuracy: %.2f' % (accuracy*100))

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 823us/step - accuracy: 0.6567 - loss: 0.6143
Accuracy: 67.23


In [46]:
#### PREDICCIONES ####

predicciones = model.predict(X_test)

# La función sigmoide nos devueve los resultados en formato probabilidad.
# Convertimos los mismos a casos, tomando como umbral 0.5
y_pred = (predicciones > 0.5).astype(int)
y_pred

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 


array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
    

In [47]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[143,   0],
       [ 86,   2]])

---

## **4.En compile, cambia la configuración del loss para usar como función de pérdida CategoricalCrossentropy.**

tf.keras.losses.CategoricalCrossentropy(
    from_logits=False,
    label_smoothing=0.0,
    axis=-1,
    reduction="auto",
    name="categorical_crossentropy"),


In [48]:
model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.1, nesterov=False, name='SGD'),
    loss = tf.keras.losses.CategoricalCrossentropy(
        from_logits=False,
        label_smoothing=0.0,
        axis=-1,
        # reduction="auto",
        name="categorical_crossentropy"
    ),
    metrics=['accuracy']  # Métrica de precisión
)

In [49]:
# seed(1)
# random.set_seed(1)
# El argumento batch_size, nos permite definir el número de filas que se considerarán, antes de que los pesos del
# modelo se reajusten dentro de cada ciclo.
model.fit(X_train, y_train, epochs=150, batch_size=10)

# En el aprendizaje automático, entrenar modelos de aprendizaje profundo con grandes conjuntos de datos puede 
# requerir mucha memoria y recursos computacionales. El uso de un tamaño de lote adecuado puede ayudar a evitar 
# problemas de memoria y a mejorar la velocidad de entrenamiento.

# Por ejemplo, si tenemos un conjunto de datos de 1000 muestras y establecemos batch_size=10, esto significa 
# que se utilizarán 10 muestras a la vez para actualizar los pesos del modelo en cada iteración del entrenamiento. 
# Por lo tanto, el proceso de entrenamiento se dividirá en 100 iteraciones, una para cada lote de 10 muestras.

# El tamaño de lote adecuado depende del conjunto de datos y del modelo. En general, un tamaño de lote más grande
# puede proporcionar un mejor rendimiento de entrenamiento, pero requiere más memoria. Por otro lado, un tamaño de
# lote más pequeño puede proporcionar una mejor generalización del modelo, pero puede requerir más iteraciones para
# converger a una solución óptima.

# En resumen, batch_size es un parámetro importante en el proceso de entrenamiento de modelos de aprendizaje profundo en Keras y su elección puede afectar el rendimiento del modelo y la velocidad de entrenamiento.

Epoch 1/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 980us/step - accuracy: 0.6489 - loss: 0.0000e+00
Epoch 2/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 893us/step - accuracy: 0.6571 - loss: 0.0000e+00
Epoch 3/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 884us/step - accuracy: 0.6572 - loss: 0.0000e+00
Epoch 4/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6572 - loss: 0.0000e+00
Epoch 5/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 789us/step - accuracy: 0.6572 - loss: 0.0000e+00
Epoch 6/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 769us/step - accuracy: 0.6572 - loss: 0.0000e+00
Epoch 7/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 830us/step - accuracy: 0.6568 - loss: 0.0000e+00
Epoch 8/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 958us/step - accuracy: 0.6572 - loss: 0.0000e+00
Epoch 9/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 877us/step - accuracy: 0.6572 - loss: 0.0000e+00
Epoch 10/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 859us/step - accuracy: 0.6572 - loss: 0.0000e+00
Epoch 11/150
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 914us/step - accuracy: 0.6572 - loss: 0.0000e+00

In [50]:
#### EVALUAMOS MODELO

# Con la red neuronal entrenada, ahora debemos evaluar cómo ha funcionado.
_, accuracy = model.evaluate(X_train, y_train)
print('Accuracy: %.2f' % (accuracy*100))

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6511 - loss: 0.0000e+00
Accuracy: 66.48


In [51]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[143,   0],
       [ 86,   2]])

---

## **5.Prueba diferentes configuraciones a ver si consigues mejorar el resultado inicial.**

### . . . —> **Opción 1**

#### Creamos una función para construir y entrenar el modelo con configuraciones personalizadas. Luego, iteraremos sobre varias configuraciones.

In [52]:
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
from sklearn.metrics import accuracy_score

# Función para construir y evaluar el modelo
def build_and_evaluate_model(hidden_layer_sizes, activation, learning_rate, batch_size, epochs):
    # Construir el modelo
    model = Sequential()
    model.add(Dense(hidden_layer_sizes[0], activation=activation, input_dim=8))
    for neurons in hidden_layer_sizes[1:]:
        model.add(Dense(neurons, activation=activation))
    model.add(Dense(1, activation='sigmoid'))  # Capa de salida

    # Compilar el modelo
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    # Entrenar el modelo
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

    # Evaluar el modelo en el conjunto de prueba
    y_pred = (model.predict(X_test) > 0.5).astype(int)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Config: {hidden_layer_sizes}, {activation}, LR: {learning_rate}, Batch: {batch_size}, Epochs: {epochs} -> Accuracy: {accuracy:.2f}")
    return accuracy


#### Probar diferentes configuraciones

Ahora probamos varias combinaciones de hiperparámetros para encontrar la mejor.


In [ ]:
# Configuraciones para probar
hidden_layer_configs = [[12, 8], [16, 8, 4], [32, 16], [64, 32, 16]]
activations = ['relu', 'tanh']
learning_rates = [0.01, 0.001]
batch_sizes = [10, 32, 64]
epochs_list = [50, 100, 150]

# Resultados
best_accuracy = 0
best_config = None

for hidden_layers in hidden_layer_configs:
    for activation in activations:
        for lr in learning_rates:
            for batch_size in batch_sizes:
                for epochs in epochs_list:
                    accuracy = build_and_evaluate_model(hidden_layers, activation, lr, batch_size, epochs)
                    if accuracy > best_accuracy:
                        best_accuracy = accuracy
                        best_config = (hidden_layers, activation, lr, batch_size, epochs)

print(f"Mejor configuración: {best_config} -> Accuracy: {best_accuracy:.2f}")


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
Config: [12, 8], relu, LR: 0.01, Batch: 10, Epochs: 50 -> Accuracy: 0.76
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Config: [12, 8], relu, LR: 0.01, Batch: 10, Epochs: 100 -> Accuracy: 0.78
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
Config: [12, 8], relu, LR: 0.01, Batch: 10, Epochs: 150 -> Accuracy: 0.62
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
Config: [12, 8], relu, LR: 0.01, Batch: 32, Epochs: 50 -> Accuracy: 0.71
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
Config: [12, 8], relu, LR: 0.01, Batch: 32, Epochs: 100 -> Accuracy: 0.81
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Config: [12, 8], relu, LR: 0.01, Batch: 32, Epochs: 150 -> Accuracy: 0.70
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
Config: [12, 8], relu, LR: 0.01, Batch: 64, Epochs: 50 -> Accuracy: 0.68
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Config: [12, 8], relu, LR: 0.01, Batch: 64, Epochs: 100 -> Accuracy: 0.71
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Config: [12, 8], relu, LR: 0.01, Batch: 64, Epochs: 150 -> Accuracy: 0.7

Mejor configuración: ([12, 8], 'relu', 0.01, 32, 100) -> Accuracy: 0.81

### Establecemos modelo con mejor conficuración

In [ ]:
# Definiremos el modelo como una secuencia de capas.
# Usaremos el modelo secuencial de manera que podamos ir añadiendo capas hasta estar contentos con la arquitectura desarrollada.
model = Sequential()

# Partimos de un sistema con 8 variables por lo que nuestra primera capa acomodará dichas variables
# En la primera capa oculta usaremos 12 neuronas y una función de activación ReLU tal y como nos lo 
# En la segunda capa oculta usaremos 8 neuronas y una función de activación ReLU
# Finalmente en la de salida una neurona y función de activación sigmoide
model.add(Dense(12, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Nota: Fíjate que el total de neuronas de entrada, lo definimos en la primera capa con input_dim

In [98]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
        #loss="binary_crossentropy", por que es una clasificación binaria
        loss=tf.keras.losses.BinaryCrossentropy(
            from_logits=False, # usar cuando valores de predicción sean [0,1]
            label_smoothing=0.0,
            axis=-1,
            name="binary_crossentropy"
        ), 
    metrics=['accuracy'] # Métrica eleginda
)


In [103]:
model.fit(X_train, y_train, epochs=100, batch_size=32)


Epoch 1/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7694 - loss: 0.4632
Epoch 2/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7841 - loss: 0.4501
Epoch 3/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7796 - loss: 0.4523
Epoch 4/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7852 - loss: 0.4524
Epoch 5/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7863 - loss: 0.4559
Epoch 6/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7890 - loss: 0.4578
Epoch 7/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7890 - loss: 0.4593
Epoch 8/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7890 - loss: 0.4598
Epoch 9/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7890 - loss: 0.4613
Epoch 10/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7887 - loss: 0.4613
Epoch 11/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7886 - loss: 0.4621
Epoch 12/100
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy:

In [104]:
_, accuracy_train = model.evaluate(X_train, y_train, verbose=0)
_, accuracy_test  = model.evaluate(X_test,  y_test,  verbose=0)

print("Training accuracy: {:.2f}%".format(accuracy_train*100))
print("Test accuracy: {:.2f}%".format(accuracy_test*100))


Training accuracy: 78.58%
Test accuracy: 78.35%


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

# ☝️👀

---

### . . . —> **Opción 2**

#### Creamos una función (más configurada que la anterior) para construir y entrenar el modelo con configuraciones personalizadas. Luego, iteraremos sobre varias configuraciones.

In [54]:
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
from sklearn.metrics import accuracy_score

# Función para construir y evaluar el modelo
def build_and_evaluate_model(hidden_layer_sizes, activation, optimizer_name, loss_name, learning_rate, batch_size, epochs):
    """
    Construye, entrena y evalúa un modelo secuencial de Keras.
    
    Parámetros:
    -----------
    hidden_layer_sizes: list
        Lista con el número de neuronas para cada capa oculta, e.g. [12, 8] o [16, 8, 4].
    activation: str
        Función de activación para capas ocultas (e.g. 'relu', 'tanh').
    optimizer_name: str
        Nombre del optimizador a usar (e.g. 'adam', 'sgd', 'rmsprop').
    loss_name: str
        Función de pérdida a usar (e.g. 'binary_crossentropy', 'hinge').
    learning_rate: float
        Tasa de aprendizaje.
    batch_size: int
        Tamaño de batch para entrenar.
    epochs: int
        Número de épocas de entrenamiento.
    
    Retorna:
    --------
    accuracy: float
        Exactitud en el conjunto de prueba.
    """
    
    # 1) Crear el modelo
    model = Sequential()
    # Capa de entrada y primera capa oculta
    model.add(Dense(hidden_layer_sizes[0], activation=activation, input_dim=8))
    
    # Resto de capas ocultas
    for neurons in hidden_layer_sizes[1:]:
        model.add(Dense(neurons, activation=activation))
        
    # Capa de salida
    model.add(Dense(1, activation='sigmoid'))
    
    # 2) Definir el optimizador
    if optimizer_name.lower() == 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer_name.lower() == 'sgd':
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer_name.lower() == 'rmsprop':
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    else:
        # Si tuviéramos más optimizadores que quisieras probar, los agremgamos aquí
        raise ValueError(f"Optimizador {optimizer_name} no reconocido.")
    
    # 3) Compilar el modelo
    model.compile(
        optimizer=optimizer,
        loss=loss_name,
        metrics=['accuracy']
    )
    
    # 4) Entrenar el modelo
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)
    
    # 5) Evaluar el modelo en el conjunto de prueba
    y_pred = (model.predict(X_test) > 0.5).astype(int)
    accuracy = accuracy_score(y_test, y_pred)
    
    config_string = (f"hidden={hidden_layer_sizes}, act={activation}, "
                     f"opt={optimizer_name}, loss={loss_name}, "
                     f"LR={learning_rate}, batch={batch_size}, epochs={epochs}")
    
    print(f"{config_string} -> Accuracy: {accuracy:.3f}")
    
    return accuracy

#### Probar diferentes configuraciones

In [ ]:
##########################################
##### CUIDADO AL EJECUTAR: 40 MINUTOS#####
##########################################

# Hiperparámetros para probar
hidden_layer_configs = [
    [12, 8], 
    [16, 8, 4], 
    [32, 16], 
    [64, 32, 16]
]
activations = ['relu', 'tanh']
optimizers = ['adam', 'sgd', 'rmsprop']
losses = ['binary_crossentropy', 'hinge']  # hinge es otra opción, aunque typical para SVM
learning_rates = [0.01, 0.001]
batch_sizes = [10, 32, 64]
epochs_list = [50, 100]

best_accuracy = 0
best_config = None

for hidden_layers in hidden_layer_configs:
    for activation in activations:
        for optimizer_name in optimizers:
            for loss_name in losses:
                for lr in learning_rates:
                    for batch_size in batch_sizes:
                        for epochs in epochs_list:
                            accuracy = build_and_evaluate_model(
                                hidden_layer_sizes=hidden_layers,
                                activation=activation,
                                optimizer_name=optimizer_name,
                                loss_name=loss_name,
                                learning_rate=lr,
                                batch_size=batch_size,
                                epochs=epochs
                            )
                            # Comprobamos si mejora lo anterior
                            if accuracy > best_accuracy:
                                best_accuracy = accuracy
                                best_config = (hidden_layers, activation, optimizer_name, loss_name, lr, batch_size, epochs)

print("=====================================================")
print(f"Mejor configuración: {best_config} -> Accuracy: {best_accuracy:.3f}")

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
hidden=[12, 8], act=relu, opt=adam, loss=binary_crossentropy, LR=0.01, batch=10, epochs=50 -> Accuracy: 0.758
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
hidden=[12, 8], act=relu, opt=adam, loss=binary_crossentropy, LR=0.01, batch=10, epochs=100 -> Accuracy: 0.619
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
hidden=[12, 8], act=relu, opt=adam, loss=binary_crossentropy, LR=0.01, batch=32, epochs=50 -> Accuracy: 0.779
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
hidden=[12, 8], act=relu, opt=adam, loss=binary_crossentropy, LR=0.01, batch=32, epochs=100 -> Accuracy: 0.697
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
hidden=[12, 8], act=relu, opt=adam, loss=binary_crossentropy, LR=0.01, batch=64, epochs=50 -> Accuracy: 0.697
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
hidden=[12, 8], act=relu, opt=adam, loss=binary_crossentropy, LR=0.01, batch=64, epochs=100 -> Accuracy: 0.723
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
hidden=[12, 8], act=relu, opt=adam, loss=binary_crossentropy, LR=0.001,

### Generamos modelo final aplicando los _**hiperparametros de la mejor configuración sugerida**_ en la búsqueda de la celda anterior

hidden=[12, 8], act=relu, opt=adam, loss=binary_crossentropy, LR=0.01, batch=32, epochs=50 -> Accuracy: 0.779

In [85]:
# Definiremos el modelo como una secuencia de capas.
# Usaremos el modelo secuencial de manera que podamos ir añadiendo capas hasta estar contentos con la arquitectura desarrollada.
model = Sequential()

# Partimos de un sistema con 8 variables por lo que nuestra primera capa acomodará dichas variables
# En la primera capa oculta usaremos 12 neuronas y una función de activación ReLU tal y como nos lo 
# En la segunda capa oculta usaremos 8 neuronas y una función de activación ReLU
# Finalmente en la de salida una neurona y función de activación sigmoide
model.add(Dense(12, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Nota: Fíjate que el total de neuronas de entrada, lo definimos en la primera capa con input_dim


In [93]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
        #loss="binary_crossentropy", por que es una clasificación binaria
        loss=tf.keras.losses.BinaryCrossentropy(
            from_logits=False, # usar cuando valores de predicción sean [0,1]
            label_smoothing=0.0,
            axis=-1,
            name="binary_crossentropy"
        ), 
    metrics=['accuracy'] # Métrica eleginda
)


In [94]:
model.fit(X_train, y_train, epochs=32, batch_size=50)


Epoch 1/32
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6376 - loss: 2.0407
Epoch 2/32
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6200 - loss: 1.1703 
Epoch 3/32
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7142 - loss: 0.7074
Epoch 4/32
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7440 - loss: 0.5779
Epoch 5/32
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6964 - loss: 0.6320
Epoch 6/32
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6853 - loss: 0.6433
Epoch 7/32
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7134 - loss: 0.5943
Epoch 8/32
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7283 - loss: 0.5554
Epoch 9/32
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7482 - loss: 0.4984
Epoch 10/32
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7688 - loss: 0.4904
Epoch 11/32
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7158 - loss: 0.5383
Epoch 12/32
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7280 - l

In [95]:
#### EVALUAMOS MODELO

# Con la red neuronal entrenada, ahora debemos evaluar cómo ha funcionado.
_, accuracy = model.evaluate(X_train, y_train)
print('Accuracy: %.2f' % (accuracy*100))

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7392 - loss: 0.5268
Accuracy: 78.40


In [97]:
_, accuracy_train = model.evaluate(X_train, y_train, verbose=0)
_, accuracy_test  = model.evaluate(X_test,  y_test,  verbose=0)

print("Training accuracy: {:.2f}%".format(accuracy_train*100))
print("Test accuracy: {:.2f}%".format(accuracy_test*100))


Training accuracy: 78.40%
Test accuracy: 74.89%


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

# ✨ 🏁 